In [76]:
import pandas as pd
import csv
import matplotlib as plot

In [77]:
deadout_df = pd.read_csv("bee_data/bee_loss_deadout.csv")
collapse_df = pd.read_csv("bee_data/colony_collapse_bees.csv")
pesticides_df = pd.read_csv("bee_data/pesticides_death_bees.csv")

deadout_df.dropna(axis=1, how="all").copy()
collapse_df.dropna(axis=1, how="all").copy()
pesticides_df.dropna(axis=1, how="all").copy()

,Program,Year,Period,Geo Level,State,State ANSI,watershed_code,Commodity,Data Item,Domain,Domain Category,Value
0,SURVEY,2020,JAN THRU MAR,NATIONAL,US TOTAL,NaN,0,HONEY,"HONEY, BEE COLONIES, AFFECTED BY PESTICIDES - ...",TOTAL,NOT SPECIFIED,5
1,SURVEY,2020,JAN THRU MAR,STATE,ALABAMA,1.0,0,HONEY,"HONEY, BEE COLONIES, AFFECTED BY PESTICIDES - ...",TOTAL,NOT SPECIFIED,0.1
2,SURVEY,2020,JAN THRU MAR,STATE,ARIZONA,4.0,0,HONEY,"HONEY, BEE COLONIES, AFFECTED BY PESTICIDES - ...",TOTAL,NOT SPECIFIED,0
3,SURVEY,2020,JAN THRU MAR,STATE,ARKANSAS,5.0,0,HONEY,"HONEY, BEE COLONIES, AFFECTED BY PESTICIDES - ...",TOTAL,NOT SPECIFIED,2.6
4,SURVEY,2020,JAN THRU MAR,STATE,CALIFORNIA,6.0,0,HONEY,"HONEY, BEE COLONIES, AFFECTED BY PESTICIDES - ...",TOTAL,NOT SPECIFIED,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...
985,SURVEY,2015,OCT THRU DEC,STATE,VIRGINIA,51.0,0,HONEY,"HONEY, BEE COLONIES, AFFECTED BY PESTICIDES - ...",TOTAL,NOT SPECIFIED,0
986,SURVEY,2015,OCT THRU DEC,STATE,WASHINGTON,53.0,0,HONEY,"HONEY, BEE COLONIES, AFFECTED BY PESTICIDES - ...",TOTAL,NOT SPECIFIED,0.3
987,SURVEY,2015,OCT THRU DEC,STATE,WEST VIRGINIA,54.0,0,HONEY,"HONEY, BEE COLONIES, AFFECTED BY PESTICIDES - ...",TOTAL,NOT SPECIFIED,3
988,SURVEY,2015,OCT THRU DEC,STATE,WISCONSIN,55.0,0,HONEY,"HONEY, BEE COLONIES, AFFECTED BY PESTICIDES - ...",TOTAL,NOT SPECIFIED,21.9


In [78]:
deadout_df["Value"] = deadout_df["Value"].str.replace(",","")
deadout_df['Value'] = deadout_df['Value'].astype('int64')
deadout = deadout_df.groupby(['Year', 'State',], as_index=False)[['Value']].sum()
deadout = deadout.rename(columns = {
    'Year': 'year',
    'State': 'state',
    'Value': 'deadout'
    })
deadout

,year,state,deadout
0,2015,ALABAMA,4670
1,2015,ARIZONA,28100
2,2015,ARKANSAS,14800
3,2015,CALIFORNIA,584000
4,2015,COLORADO,10400
...,...,...,...
277,2020,VIRGINIA,2300
278,2020,WASHINGTON,4600
279,2020,WEST VIRGINIA,1670
280,2020,WISCONSIN,3200


In [79]:
collapse_df["Value"] = collapse_df["Value"].str.replace(",","")
collapse_df['Value'] = collapse_df['Value'].astype('int64')
collapse = collapse_df.groupby(['Year', 'State',], as_index=False)[['Value']].sum()
collapse = collapse.rename(columns = {
    'Year': 'year',
    'State': 'state',
    'Value': 'cc_syn'
    })
collapse

,year,state,cc_syn
0,2015,US TOTAL,332010
1,2016,US TOTAL,391130
2,2017,US TOTAL,251130
3,2018,US TOTAL,330680
4,2019,ALABAMA,430
...,...,...,...
92,2020,VIRGINIA,540
93,2020,WASHINGTON,6870
94,2020,WEST VIRGINIA,330
95,2020,WISCONSIN,1690


In [80]:
pesticides_df.drop(pesticides_df.loc[pesticides_df['Value'] == ' (Z)'].index, inplace=True)
pesticides_df['Value'] = pesticides_df['Value'].astype('float64')
pesticides_df = pesticides_df.loc[pesticides_df['Domain'] == 'TOTAL']
pesticides = pesticides_df.groupby(['Year', 'State',], as_index=False)[['Value']].sum()
pesticides = pesticides.rename(columns = {
    'Year': 'year',
    'State': 'state',
    'Value': 'pesticides'
    })
pesticides

,year,state,pesticides
0,2015,ALABAMA,7.3
1,2015,ARIZONA,40.9
2,2015,ARKANSAS,39.2
3,2015,CALIFORNIA,61.8
4,2015,COLORADO,39.6
...,...,...,...
277,2020,VIRGINIA,0.7
278,2020,WASHINGTON,1.5
279,2020,WEST VIRGINIA,1.1
280,2020,WISCONSIN,25.1


In [81]:
combined = deadout.merge(collapse, on=['state','year'])
combined_final = combined.merge(pesticides, on=['state','year'])
combined_final

,year,state,deadout,cc_syn,pesticides
0,2015,US TOTAL,1756860,332010,48.7
1,2016,US TOTAL,1674960,391130,47.8
2,2017,US TOTAL,1534610,251130,47.1
3,2018,US TOTAL,1615150,330680,53.1
4,2019,ALABAMA,2300,430,5.4
...,...,...,...,...,...
92,2020,VIRGINIA,2300,540,0.7
93,2020,WASHINGTON,4600,6870,1.5
94,2020,WEST VIRGINIA,1670,330,1.1
95,2020,WISCONSIN,3200,1690,25.1
